In [16]:
import slicer, vtk
import numpy as np
import vtk.util.numpy_support
from scipy import optimize
from DICOMLib import DICOMUtils
import os
import pandas as pd
import sys
sys.path.append('../')
from config import load_config


In [17]:
config = load_config(os.path.abspath("../config.yml"))
df_path = config.DATA_FILE
DICOM_path = config.DICOM_PATH
voxelized_save_dir = config.VOXELIZED_SAVE_PATH
mrb_save_dir = config.MRB_SAVE_PATH
os.makedirs(os.path.join(mrb_save_dir), exist_ok=True)

In [18]:
def load_DICOM(p):
    slicer.mrmlScene.Clear(0)


    dicomDataDir = os.path.join(DICOM_path, p)

    loadedNodeIDs = []  # this list will contain the list of all loaded node IDs

    with DICOMUtils.TemporaryDICOMDatabase() as db:
        DICOMUtils.importDicom(dicomDataDir, db)
        patientUIDs = db.patients()
        for patientUID in patientUIDs:
            loadedNodeIDs.extend(DICOMUtils.loadPatientByUID(patientUID))
    
def deleteBeams():
    i = 0
    while len(slicer.util.getNodesByClass("vtkMRMLSequenceNode")) > 0:
        i += 1
        toDeletes = slicer.util.getNodesByClass("vtkMRMLSequenceNode")
        for td in toDeletes:
            slicer.mrmlScene.RemoveNode(td)
        if i > 10:
            break

    while len(slicer.util.getNodesByClass("vtkMRMLSequenceBrowserNode")) > 0:
        i += 1
        toDeletes = slicer.util.getNodesByClass("vtkMRMLSequenceBrowserNode")
        for td in toDeletes:
            slicer.mrmlScene.RemoveNode(td)
        if i > 10:
            break

    nodes = slicer.util.getNodes()
    to_remove = []
    for key in nodes.keys():
        #if "[TREATMENT]" in key:
        #if "cw" in key or "CW" in key or (len(key) == 12 and key[4] == "_") or "BoundaryAndPosition" in key or "BeamTransform" in key or "[TREATMENT]" in key:
        #if "cw" in key or "CW" in key:
        if "BeamTransform" in key:
        #if "cw" in key or "CW" in key or (len(key) == 12 and key[4] == "_"):
            #print(key)
            to_remove.append(nodes[key])
            if type(nodes[key]).__name__ == "vtkMRMLRTBeamNode":
                if nodes[key].GetModelDisplayNode() is not None:
                    to_remove.append(nodes[key].GetModelDisplayNode())
                if nodes[key].GetParentTransformNode() is not None:
                    to_remove.append(nodes[key].GetParentTransformNode())
                if nodes[key].GetStorageNode() is not None:
                    to_remove.append(nodes[key].GetStorageNode())
            if type(nodes[key]).__name__ == "vtkMRMLTableNode":
                if nodes[key].GetStorageNode() is not None:
                    to_remove.append(nodes[key].GetStorageNode())


    for n in to_remove:
        slicer.mrmlScene.RemoveNode(n)


    nodes = slicer.util.getNodes()
    to_remove = []
    for key in nodes.keys():
        #if "[TREATMENT]" in key:
        if "cw" in key or "CW" in key or "SA " in key or (len(key) == 12 and key[4] == "_") or "BoundaryAndPosition" in key or "BeamTransform" in key or "[TREATMENT]" in key:
        #if "cw" in key or "CW" in key:
        #if "BeamTransform" in key:
        #if "cw" in key or "CW" in key or (len(key) == 12 and key[4] == "_"):
            #print(key)
            to_remove.append(nodes[key])
            if type(nodes[key]).__name__ == "vtkMRMLRTBeamNode":
                if nodes[key].GetModelDisplayNode() is not None:
                    to_remove.append(nodes[key].GetModelDisplayNode())
                if nodes[key].GetParentTransformNode() is not None:
                    to_remove.append(nodes[key].GetParentTransformNode())
                if nodes[key].GetStorageNode() is not None:
                    to_remove.append(nodes[key].GetStorageNode())
            if type(nodes[key]).__name__ == "vtkMRMLTableNode":
                if nodes[key].GetStorageNode() is not None:
                    to_remove.append(nodes[key].GetStorageNode())

    #break
    #print(to_remove)
    #break

    for n in to_remove:
        slicer.mrmlScene.RemoveNode(n)
        
    while len(slicer.util.getNodesByClass("vtkMRMLRTPlanNode")) > 0:
        i += 1
        toDeletes = slicer.util.getNodesByClass("vtkMRMLRTPlanNode")
        for td in toDeletes:
            slicer.mrmlScene.RemoveNode(td)
        if i > 10:
            break


In [19]:
def copySegment(inputId, outputId, segmentationNode):
    segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
    segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
    segmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
    segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)

    segmentEditorWidget.setSegmentationNode(segmentationNode)
    volumeNode = slicer.util.getNodesByClass("vtkMRMLVolumeNode")[0]
    segmentEditorWidget.setMasterVolumeNode(volumeNode)
    # Set overwrite mode: 0/1/2 -> overwrite all/visible/none
    segmentEditorNode.SetOverwriteMode(2) # i.e. "allow overlap" in UI
    # Get the segment IDs
    segmentationNode.AddSegmentFromClosedSurfaceRepresentation(vtk.vtkPolyData(), outputId)
    segid_tgt = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(outputId)
    segid_src = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(inputId)
    
    segmentEditorNode.SetSelectedSegmentID(segid_tgt)
    segmentEditorWidget.setActiveEffectByName("Logical operators")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("Operation","COPY") # change the operation here
    effect.setParameter("ModifierSegmentID",segid_src)
    effect.self().onApply()
    
    slicer.mrmlScene.RemoveNode(segmentEditorNode)
    
def combineSegments(inputId, outputId, segmentationNode):
    segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
    segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
    segmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
    segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
    
    segmentEditorWidget.setSegmentationNode(segmentationNode)
    volumeNode = slicer.util.getNodesByClass("vtkMRMLVolumeNode")[0]
    segmentEditorWidget.setMasterVolumeNode(volumeNode)
    # Set overwrite mode: 0/1/2 -> overwrite all/visible/none
    segmentEditorNode.SetOverwriteMode(2) # i.e. "allow overlap" in UI
    # Get the segment IDs
    segid_tgt = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(outputId)
    segid_src = segmentationNode.GetSegmentation().GetSegmentIdBySegmentName(inputId)
    
    segmentEditorNode.SetSelectedSegmentID(segid_tgt)
    segmentEditorWidget.setActiveEffectByName("Logical operators")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("Operation","UNION") # change the operation here
    effect.setParameter("ModifierSegmentID",segid_src)
    effect.self().onApply()
    
    slicer.mrmlScene.RemoveNode(segmentEditorNode)
    
def processData(p, PTVs, IGTVs):
    #Deleted Unnamed Series if exists
    print("Deleting Unnamed series")
    scalar_volumes = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")
    nodes_to_remove = []
    for i in range(len(scalar_volumes)):
        node  = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")[i]
        if "Unnamed" in node.GetName():
            nodes_to_remove.append(node)

    #Deleting volume node if not right size
    volumes = slicer.util.getNodesByClass("vtkMRMLVolumeNode")
    nodes_to_remove = []
    for i in range(len(scalar_volumes)):
        node = slicer.util.getNodesByClass("vtkMRMLVolumeNode")[i]
        if slicer.util.arrayFromVolume(node).shape[1] != 512 or slicer.util.arrayFromVolume(node).shape[0] <= 50:
            if not "RTDOSE" in node.GetName():
                nodes_to_remove.append(node)

    for n in nodes_to_remove:
        slicer.mrmlScene.RemoveNode(n)


    #Deleting existing RTStructs
    segNodes = slicer.util.getNodesByClass("vtkMRMLSegmentationNode")
    for s in segNodes:
        slicer.mrmlScene.RemoveNode(s)

    referenceVolumeNode = slicer.util.getNodesByClass("vtkMRMLVolumeNode")[0]
    
    print("Removing beam nodes")
    deleteBeams()
  
    newSegmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    newSegmentationNode.CreateDefaultDisplayNodes()
    newSegmentation = newSegmentationNode.GetSegmentation()
    
    print("Loading RT structs")
    
    ##Loading RT label maps for RT structs
    voxelized_structs = np.load(os.path.join(voxelized_save_dir, p, "VoxelizedStructs.npz"))
    all_structs = voxelized_structs.files
    for struct in all_structs:
        struct_arr = voxelized_structs[struct]

        if struct == config.BODY:
            struct_arr = ~struct_arr
        
        struct_arr = struct_arr[:, :, 1:]
        struct_arr = np.moveaxis(struct_arr, [2,0,1], [0,1,2])

        labelmap_volume_Node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
        labelmap_volume_Node.CopyOrientation(referenceVolumeNode)
        labelmap_volume_Node.SetSpacing(referenceVolumeNode.GetSpacing())
        labelmap_volume_Node.CreateDefaultDisplayNodes()

        slicer.util.updateVolumeFromArray(labelmap_volume_Node, struct_arr.astype("int")[::1])
        
        slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelmap_volume_Node, newSegmentationNode)
        numberOfSegments = newSegmentation.GetNumberOfSegments()
        current_segmentID = newSegmentation.GetNthSegmentID(numberOfSegments - 1)
        copySegment(current_segmentID, struct, newSegmentationNode)
        newSegmentationNode.RemoveSegment(current_segmentID)
        
        #Do again for aorta
        if struct == config.AORTA:
            slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelmap_volume_Node, newSegmentationNode)
            numberOfSegments = newSegmentation.GetNumberOfSegments()
            current_segmentID = newSegmentation.GetNthSegmentID(numberOfSegments - 1)
            copySegment(current_segmentID, "GV_Combined", newSegmentationNode)
            newSegmentationNode.RemoveSegment(current_segmentID)
        
        #newSegmentation.GetNthSegment(numberOfSegments - 1).SetName(struct[:-4])
        
        slicer.mrmlScene.RemoveNode(labelmap_volume_Node)
        
    #Creating Combination of GreatVessels
    #print("Combining Great Vessels")
    
    for temp_struct in [config.VENACAVA_INF, config.VENACAVA_SUP, config.GREATVESSELS]:
        combineSegments(temp_struct, "GV_Combined", newSegmentationNode)


In [21]:
df = pd.read_csv(df_path)
for i in range(df.shape[0]):
        p = df.iloc[i]["Patient"]

        slicer.mrmlScene.Clear(0)

        try:
            num_lesions = df.iloc[i]["Number of Lesions"]
        except:
            num_lesions = 1
        PTVs = df.iloc[i]["PTVs"].split(",")
        IGTVs = df.iloc[i]["IGTVs"].split(",")
        try:
            assert num_lesions == len(PTVs) == len(IGTVs)
        except:
            print("assertion fail")

        load_DICOM(p)

        # processData(p, PTVs, IGTVs)
        print("Done Processing")
        #Exporting
        # sceneSaveFilename = os.path.join(mrb_save_dir, p + ".mrb")
        # if slicer.util.saveScene(sceneSaveFilename):
        #     print("Scene saved to: {0}".format(sceneSaveFilename))
        #     #logging.info("Scene saved to: {0}".format(sceneSaveFilename))
        # else:
        #     #logging.error("Scene saving failed")
        #     print("Scene saving failed")

print("end")

Done Processing
end


In [ ]:
newSegmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
newSegmentationNode.CreateDefaultDisplayNodes()
newSegmentation = newSegmentationNode.GetSegmentation()

In [15]:
##Loading RT label maps for RT structs
voxelized_structs = np.load(os.path.join(voxelized_save_dir, p, "VoxelizedStructs.npz"))
all_structs = voxelized_structs.files
for struct in all_structs:
    struct_arr = voxelized_structs[struct]

    if struct == config.BODY:
        struct_arr = ~struct_arr

    struct_arr = struct_arr[:, :, 1:]
    struct_arr = np.moveaxis(struct_arr, [2,0,1], [0,1,2])

    labelmap_volume_Node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    labelmap_volume_Node.CopyOrientation(referenceVolumeNode)
    labelmap_volume_Node.SetSpacing(referenceVolumeNode.GetSpacing())
    labelmap_volume_Node.CreateDefaultDisplayNodes()

    slicer.util.updateVolumeFromArray(labelmap_volume_Node, struct_arr.astype("int")[::1])

    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelmap_volume_Node, newSegmentationNode)
    numberOfSegments = newSegmentation.GetNumberOfSegments()
    current_segmentID = newSegmentation.GetNthSegmentID(numberOfSegments - 1)
    copySegment(current_segmentID, struct, newSegmentationNode)
    newSegmentationNode.RemoveSegment(current_segmentID)

    #Do again for aorta
    if struct == config.AORTA:
        slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelmap_volume_Node, newSegmentationNode)
        numberOfSegments = newSegmentation.GetNumberOfSegments()
        current_segmentID = newSegmentation.GetNthSegmentID(numberOfSegments - 1)
        copySegment(current_segmentID, "GV_Combined", newSegmentationNode)
        newSegmentationNode.RemoveSegment(current_segmentID)

    #newSegmentation.GetNthSegment(numberOfSegments - 1).SetName(struct[:-4])

    slicer.mrmlScene.RemoveNode(labelmap_volume_Node)
    break
        

In [25]:
referenceVolumeNode = slicer.util.getNodesByClass("vtkMRMLVolumeNode")[0]

In [24]:
testSegmentationNode = slicer.util.getNodesByClass("vtkMRMLSegmentationNode")[1]

In [22]:
sh = slicer.mrmlScene.GetSubjectHierarchyNode()

In [28]:
sh.SetItemParent(sh.GetItemByDataNode(testSegmentationNode),  sh.GetItemParent(sh.GetItemByDataNode(referenceVolumeNode)))